# 🧩 CSCI 331 – Homework 6: Set Operators (Group 1045)
**Student:** Ali  
**Database:** AdventureWorks2022  
**Focus:** Set Operators + Concepts from Chapters 1–6

### Proposition 1 – Vendors Who Are Preferred or Active
**Functional Spec:** Uses `UNION` (Ch.6) and `WHERE` (Ch.2) to combine preferred and active vendors from `Purchasing.Vendor`.

In [ ]:
SELECT Name, CreditRating, 'Preferred' AS VendorType
FROM Purchasing.Vendor
WHERE PreferredVendorStatus = 1
UNION
SELECT Name, CreditRating, 'Active' AS VendorType
FROM Purchasing.Vendor
WHERE ActiveFlag = 1
ORDER BY Name;

### Proposition 2 – Vendors Who Are Both Preferred and Active
**Functional Spec:** Uses `INTERSECT` (Ch.6) to find vendors who meet both criteria. Applies sorting with `ORDER BY` (Ch.2).

In [ ]:
SELECT Name, CreditRating
FROM Purchasing.Vendor
WHERE PreferredVendorStatus = 1
INTERSECT
SELECT Name, CreditRating
FROM Purchasing.Vendor
WHERE ActiveFlag = 1
ORDER BY CreditRating DESC;

### Proposition 3 – Vendors Registered but Inactive
**Functional Spec:** Uses `EXCEPT` (Ch.6) and filtering (Ch.2) to find vendors that exist but are not active.

In [ ]:
SELECT Name
FROM Purchasing.Vendor
EXCEPT
SELECT Name
FROM Purchasing.Vendor
WHERE ActiveFlag = 1
ORDER BY Name;

### Proposition 4 – Products Ordered but Not Yet Stocked
**Functional Spec:** Combines `EXCEPT` (Ch.6) with `WHERE` and `YEAR()` (Ch.2 & Ch.4) to find recent purchase items not in inventory.

In [ ]:
SELECT DISTINCT ProductID
FROM Purchasing.PurchaseOrderDetail
WHERE YEAR(DueDate) >= 2013
EXCEPT
SELECT ProductID
FROM Production.ProductInventory;

### Proposition 5 – Products Appearing in Both Purchase Orders and Inventory
**Functional Spec:** Uses `INTERSECT` (Ch.6) and `JOIN` (Ch.3) to find overlapping product records.

In [ ]:
SELECT p.ProductID, pr.Name, pr.StandardCost, pr.ListPrice
FROM Purchasing.PurchaseOrderDetail AS p
INTERSECT
SELECT pi.ProductID, pr2.Name, pr2.StandardCost, pr2.ListPrice
FROM Production.ProductInventory AS pi
JOIN Production.Product AS pr2 ON pi.ProductID = pr2.ProductID;

### Proposition 6 – Expensive-for-Sale or Cheap-to-Produce Products
**Functional Spec:** Uses `UNION` (Ch.6) with conditional filtering and computed columns from earlier chapters.

In [ ]:
SELECT ProductID, Name, ListPrice, 'High List Price' AS Category
FROM Production.Product
WHERE ListPrice > 1000
UNION
SELECT ProductID, Name, StandardCost, 'Low Cost Product' AS Category
FROM Production.Product
WHERE StandardCost < 100
ORDER BY Name;

### Proposition 7 – In-House Manufactured Products Never Purchased
**Functional Spec:** Combines a `CTE` (Ch.5) with `EXCEPT` (Ch.6) to isolate internally produced products that were never bought externally.

In [ ]:
WITH Manufactured AS (
    SELECT ProductID, Name
    FROM Production.Product
    WHERE MakeFlag = 1
)
SELECT ProductID, Name
FROM Manufactured
EXCEPT
SELECT DISTINCT ProductID, NULL AS Name
FROM Purchasing.PurchaseOrderDetail;

### Proposition 8 – Vendors Supplying Either High or Low Order Quantities
**Functional Spec:** Uses `GROUP BY` & `HAVING` (Ch.2) with `UNION` (Ch.6) to find vendors with extreme order frequencies.

In [ ]:
SELECT VendorID, COUNT(*) AS OrderCount
FROM Purchasing.PurchaseOrderHeader
GROUP BY VendorID
HAVING COUNT(*) > 20
UNION
SELECT VendorID, COUNT(*) AS OrderCount
FROM Purchasing.PurchaseOrderHeader
GROUP BY VendorID
HAVING COUNT(*) < 3;

### Proposition 9 – Discontinued Products Still Existing in Inventory
**Functional Spec:** Uses `INTERSECT` (Ch.6) and `IS NOT NULL` (Ch.1) to identify discontinued items still physically stored.

In [ ]:
SELECT ProductID
FROM Production.Product
WHERE DiscontinuedDate IS NOT NULL
INTERSECT
SELECT ProductID
FROM Production.ProductInventory;

### Proposition 10 – Active Products Never Purchased
**Functional Spec:** Uses `EXCEPT` (Ch.6), `IS NULL` (Ch.1), and date validation (Ch.4) to find sellable items that have no purchase record.

In [ ]:
SELECT ProductID, Name
FROM Production.Product
WHERE DiscontinuedDate IS NULL AND SellStartDate <= GETDATE()
EXCEPT
SELECT DISTINCT ProductID, NULL AS Name
FROM Purchasing.PurchaseOrderDetail;